### 0. Use Google Colab

In [1]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -U tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 8.2 MB/s eta 0:00:00


### 1. Import libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt

# from drive.MyDrive.CV_project.src.ViT import ViT_classifier
from src.ViT import ViT_classifier

In [4]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

### 2. Data loading and preparation

In [5]:
mode = "coarse"
# mode = "fine"

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(
    label_mode=mode
)

169001437/169001437 [==============================] - 4s 0us/step


### 3. Train ViT

#### Hyperparameters

In [6]:
if mode == "fine":
    num_classes = 100
elif mode == "coarse":
    num_classes = 20

params = {
    "learning_rate": 0.001,
    "weight_decay": 0.0001,
    "MLP_dropout_rate": 0.5,
    "TB_dropout_rate": 0.6,
    "batch_size": 256,
    "num_epochs": 100,
    "input_shape": (32, 32, 3),
    "num_classes": num_classes,
    "image_size": 72,  # We'll resize input images to this size
    "patch_size": 6,  # Size of the patches to be extract from the input images
    "projection_dim": 64,
    "num_heads": 4,
    "transformer_units": [128, 64],  # Size of the transformer layers
    "transformer_layers": 8,
    "mlp_head_units": [2048, 1024],  # Size of the dense layers of the final classifier
}

#### Training

In [7]:
ViT = ViT_classifier(params=params)
history = ViT.train(x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

Epoch 1/100
176/176 [==============================] - 87s 367ms/step - loss: 2.8707 - accuracy: 0.1526 - top-5-accuracy: 0.4769 - val_loss: 2.4787 - val_accuracy: 0.2430 - val_top-5-accuracy: 0.6182
Epoch 2/100
176/176 [==============================] - 66s 374ms/step - loss: 2.5313 - accuracy: 0.2158 - top-5-accuracy: 0.5852 - val_loss: 2.3435 - val_accuracy: 0.2816 - val_top-5-accuracy: 0.6604
Epoch 3/100
176/176 [==============================] - 66s 374ms/step - loss: 2.4170 - accuracy: 0.2530 - top-5-accuracy: 0.6343 - val_loss: 2.2357 - val_accuracy: 0.3186 - val_top-5-accuracy: 0.6970
Epoch 4/100
176/176 [==============================] - 66s 373ms/step - loss: 2.3276 - accuracy: 0.2809 - top-5-accuracy: 0.6674 - val_loss: 2.1932 - val_accuracy: 0.3364 - val_top-5-accuracy: 0.7158
Epoch 5/100
176/176 [==============================] - 66s 375ms/step - loss: 2.2686 - accuracy: 0.2976 - top-5-accuracy: 0.6857 - val_loss: 2.1438 - val_accuracy: 0.3498 - val_top-5-accuracy: 0.7320


#### Save model

In [8]:
ViT.model.save(f"drive/MyDrive/CV_project/ViT_model_{mode}")

In [ ]:
plt.plot(history)

### 5. Show results

In [ ]:
file = "drive/MyDrive/CV_Transformer/cifar-100-python/meta"


def unpickle(file):
    import pickle

    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


dict = unpickle(file)
fine_label_names = dict[b"fine_label_names"]
coarse_label_names = dict[b"coarse_label_names"]

In [ ]:
model = tf.keras.models.load_model(f"ViT_model_{mode}")